# Dependencies

In [1]:
Sys.setlocale(category = "LC_ALL", locale = "en_US.UTF-8")

[1] "en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/C"

In [2]:
pacman::p_load(dplyr, ggplot2, ComplexHeatmap, circlize)

# Functions

In [3]:
source("/Users/anabbi/git/ped_CapTCRseq/R/ggplot2_theme.R")
source("/Users/anabbi/git/ped_CapTCRseq/R/color_schemes.R")
source("/Users/anabbi/git/ped_CapTCRseq/R/Misc_functions.R")

In [35]:
heatmap_Tcells.fx <- function(metadata_div, patient) {
    mypt <- metadata_div[metadata_div$Patient == patient, ]
    mypt_pbmc <- mypt[which(mypt$sampletype == "PBMC"), ]

    rownames(mypt_pbmc) <- mypt_pbmc$cycle

    Tcellmat <- as.matrix(mypt_pbmc[, c("Naïve.", "SCM.", "CM.", "EM.", "TE.")])
    colnames(Tcellmat) <- c("%Naïve", "%SCM", "%CM", "%EM", "%TE")

    myroworder <- c("X01", "X02", "X03", "X04", "X05")
    Tcellmat <- Tcellmat[intersect(myroworder, rownames(Tcellmat)), ] # reorder rows from cycle 1 to 5

    anno_CD3 <- HeatmapAnnotation(
        TEfold = anno_barplot((mypt_pbmc$FOLD), height = unit(2, "cm")),
        AbsoluteCount = anno_barplot((mypt_pbmc$ATC), height = unit(2, "cm")),
        TotalCD3 = anno_barplot((mypt_pbmc$CD3), height = unit(2, "cm")),
        gap = unit(2, "mm")
    )

    col_fun <- colorRamp2(c(0, 50, 100), c("white", "#ef6548", "#7f0000"))

    heatmap_tcells <- Heatmap(t(Tcellmat),
        top_annotation = anno_CD3,
        name = "T-cells %",
        #column_title = paste(unique(mypt_pbmc$Patient), unique(mypt_pbmc$Disease_type), sep = "_"),
        col = col_fun,
        # format rows
        row_names_gp = gpar(fontsize = 15),
        cluster_rows = FALSE,
        show_row_dend = FALSE,
        # format cols
        cluster_columns = FALSE,
        column_names_gp = gpar(fontsize = 15),
        rect_gp = gpar(col = "black"),
        # legend
        show_heatmap_legend = TRUE,
        width = unit(nrow(Tcellmat), "cm"),
        height = unit(2.5, "cm"),
        heatmap_legend_param = list(labels_gp = gpar(fontsize = 12))
    )
    return(heatmap_tcells)
}


In [36]:
heatmap_NCR.fx <- function(metadata_div, patient) {
    
    mypt <- metadata_div[metadata_div$Patient == patient, ]
    mypt_pbmc <- mypt[which(mypt$sampletype == "PBMC"), ]

    rownames(mypt_pbmc) <- mypt_pbmc$cycle

    NCRmat <- as.matrix(mypt_pbmc[, c("PD1.", "PDL1.", "TIM3.", "LAG3.", "CTLA4.")])
    colnames(NCRmat) <- c("%PD-1", "%PD-L1", "%TIM-3", "%LAG-3", "%CTLA-4")
    
    myroworder <- c("X01", "X02", "X03", "X04", "X05")
    NCRmat <- NCRmat[intersect(myroworder, rownames(NCRmat)), ] # reorder rows from cycle 1 to 5

    col_fun <- colorRamp2(c(0, 50, 100), c("white", "#ef6548", "#7f0000"))

    heatmap_ncr <- Heatmap(t(NCRmat),
        name = "NCR %",
        # column_title = paste(unique(mypt_pbmc$Patient), unique(mypt_pbmc$Disease_type), sep = "_"),
        col = col_fun,
        # format rows
        row_names_gp = gpar(fontsize = 15),
        cluster_rows = FALSE,
        show_row_dend = FALSE,
        # format cols
        cluster_columns = FALSE,
        column_names_gp = gpar(fontsize = 15),
        rect_gp = gpar(col = "black"),
        # legend
        show_heatmap_legend = TRUE,
        width = unit(nrow(NCRmat), "cm"),
        height = unit(2.5, "cm"),
        heatmap_legend_param = list(labels_gp = gpar(fontsize = 12))
    )
    return(heatmap_ncr)
}


In [37]:
tcr_plots.fx <- function(metadata_div, mypath="~/Desktop/H4H/INTERCEPT/NextSeq/", patient, pbmc_cfdna="PBMC") {
      mypt <- metadata_div[metadata_div$Patient == patient, ]
      mypt_pbmc <- mypt[which(mypt$sampletype == pbmc_cfdna), ]
      rownames(mypt_pbmc) <- mypt_pbmc$cycle

      myfilelst <- paste0(mypath, mypt_pbmc$file)

      i <- 1
      for (f in myfilelst) {
            mixcrfle <- read.table(f,
                  header = TRUE, sep = "\t",
                  stringsAsFactors = FALSE,
                  na.strings = c("", "NA")
            )
            if (i == 1) {
                  compldfle <- mixcrfle
                  compldfle <- cbind(
                        cloneno = row.names(compldfle),
                        filename = f,
                        compldfle
                  )
                  i <- i + 1
            } else {
                  compldfle1 <- mixcrfle
                  compldfle1 <- cbind(
                        cloneno = row.names(compldfle1),
                        filename = f,
                        compldfle1
                  )
                  compldfle <- rbind(compldfle, compldfle1)
                  rm(compldfle1)
            }
      }


      compldfle$filename <- as.character(compldfle$filename)

      compldfle$cloneno <- as.character(compldfle$cloneno)
      compldfle$cloneno <- formatC(compldfle$cloneno, width = nchar(max(as.numeric(compldfle$cloneno))), flag = "0")

      compldfle$filename <- gsub(mypath, "", compldfle$filename)
      compldfle$sample_id <- mypt_pbmc$sample_id[match(compldfle$filename, mypt_pbmc$file)]

      # diversity plot
      divpt <- ggplot(data = mypt_pbmc, aes(y = observed_Shannon, x = sample_id)) +
            geom_point(colour = "#030303", stat = "identity", size = 4) +
            theme(
                  axis.title.x = element_blank(),
                  axis.text.x = element_blank(),
                  axis.line = element_line(color = "black"),
                  axis.text = element_text(size = 15),
                  axis.title = element_text(size = 15),
                  legend.position = "none"
            ) +
            theme(
                  panel.grid.major = element_blank(),
                  panel.grid.minor = element_blank(),
                  panel.background = element_rect(fill = "transparent", colour = NA),
                  panel.border = element_blank(),
                  plot.margin = unit(c(0, 0, 0, 0), "cm")
            ) +
            annotation_logticks(sides = "l") +
            scale_y_continuous(
                  trans = "log10",
                  breaks = c(1, 50, 100, 150),
                  label = c(1, 50, 100, 150)
            )

      # TRB bar plot
      barpt <- ggplot(data = mypt_pbmc, aes(y = TRB, x = sample_id)) +
            geom_bar(fill = "#f3b94e", colour = "#030303", stat = "identity") +
            theme(
                  axis.title.x = element_blank(),
                  axis.text.x = element_blank(),
                  axis.line = element_line(color = "black"),
                  axis.text = element_text(size = 15),
                  axis.title = element_text(size = 15),
                  legend.position = "none"
            ) +
            theme(
                  panel.grid.major = element_blank(),
                  panel.grid.minor = element_blank(),
                  panel.background = element_rect(fill = "transparent", colour = NA),
                  panel.border = element_blank(),
                  plot.margin = unit(c(0, 0, 0, 0), "cm")
            )


      # TRB clones stacked bar plot
      stkbarpt <- ggplot(data = compldfle, aes(y = cloneFraction, x = sample_id)) +
            geom_bar(fill = "#d9d9d9", colour = "#000000", stat = "identity") +
            theme(
                  axis.title.x = element_blank(),
                  axis.text.x = element_text(angle = 45, hjust = 1),
                  axis.line = element_line(color = "black"),
                  axis.text = element_text(size = 15),
                  axis.title = element_text(size = 15),
                  legend.position = "none"
            ) +
            theme(
                  panel.grid.major = element_blank(),
                  panel.grid.minor = element_blank(),
                  panel.background = element_rect(fill = "transparent", colour = NA),
                  panel.border = element_blank(),
                  plot.margin = unit(c(0, 0, 0, 0), "cm")
            )

      return(align_plots1(divpt, barpt, stkbarpt))
}


In [38]:
anno_tcr.fx <- function(metadata_div, mypath, patient, cfDNA_PBMC, topx) {
    mypt <- metadata_div[metadata_div$Patient == patient, ]
    mypt_sample <- mypt[which(mypt$sampletype == cfDNA_PBMC), ]
    rownames(mypt_sample) <- mypt_sample$cycle

    myroworder <- c("X01", "X02", "X03", "X04", "X05")
    mypt_sample <- mypt_sample[intersect(myroworder, rownames(mypt_sample)), ] # reorder rows from cycle 1 to 5

    myfilelst <- paste0(mypath, mypt_sample$file)

    topX_list <- lapply(myfilelst, function(x) {
        mixcrfle <- read.table(x,
            header = TRUE, sep = "\t",
            stringsAsFactors = FALSE,
            na.strings = c("", "NA")
        )
        # get top X clones
        mixcrfle$cloneCount[1:topx]
    })

    names(topX_list) <- mypt_sample$basename

    output <- as.data.frame(matrix(topX_list[[1]], ncol = 1, byrow = TRUE))
    output$rows <- 1:nrow(output)

    for (i in 2:length(topX_list)) {
        out1 <- as.data.frame(matrix(topX_list[[i]], ncol = 1, byrow = TRUE))
        out1$rows <- 1:nrow(out1)
        output <- merge(output, out1, all = TRUE, by = "rows")
    }

    output$rows <- NULL
    output[is.na(output)] <- 0
    mytab <- t(prop.table(as.matrix(output), 2))


    breaks <- round(quantile(mypt_sample$observed_Shannon)) # for log10 scale

    anno_tcr <- HeatmapAnnotation(
        observed_Shannon = anno_points(log10(mypt_sample$observed_Shannon), axis_param = list(at = log10(breaks), labels = breaks), height = unit(2, "cm")),
        TRB = anno_barplot(mypt_sample$TRB, height = unit(2, "cm")),
        Top_clones = anno_barplot(mytab, height = unit(2, "cm")),
        gap = unit(2, "mm")
    )

    return(anno_tcr)
}

# Paths

In [9]:
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Plots/"
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Manifests/"
gitpath <- "/Users/anabbi/git/ped_CapTCRseq/"

In [10]:
h4hpath <- "/Users/anabbi/Desktop/H4H/INTERCEPT/"

In [23]:
mypath <- "~/Desktop/H4H/INTERCEPT/NextSeq/"

# Main

In [11]:
meta_div <- readr::read_rds(paste0(datapath, "capTCRseq/meta_div_goodsamples.rds"))

In [42]:
p1 <- "CHP_400"
cfDNA_PBMC <- "PBMC"
topx <- 10
disease <- unique(meta_div$Disease_type[meta_div$Patient == p1])

In [43]:
h1 <- heatmap_Tcells.fx(meta_div, p1)
h2 <- heatmap_NCR.fx(meta_div, p1)
anno1 <- anno_tcr.fx(meta_div, mypath, p1, cfDNA_PBMC, topx)

In [48]:
pdf(paste0(plotpath, "CHP400.pdf"), width = 10, height = 10)
draw(anno1 %v% h1 %v% h2,
    heatmap_legend_side = "right", column_title = paste(p1, disease, sep = "_"), column_title_gp = gpar(fontsize = 15)
)
dev.off()


pdf 
  2